# SQL 基礎

> 第二天

[數據交點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
%LOAD sqlite3 db=imdb.db timeout=2 shared_cache=true

## 建立本機學習環境

## 下載學習資料庫

[imdb.db](https://hahow-sqlfifty.s3-ap-northeast-1.amazonaws.com/imdb.db)

## SQLiteStudio 是能夠連結學習資料庫與撰寫 SQL 的圖形化介面軟體

- 在 Windows 安裝 SQLiteStudio
    - [InstallSQLiteStudio-3.2.1.exe](https://hahow-sqlfifty.s3-ap-northeast-1.amazonaws.com/InstallSQLiteStudio-3.2.1.exe)
- 在 macOS 安裝 SQLiteStudio
    - [InstallSQLiteStudio-3.2.1.dmg](https://hahow-sqlfifty.s3-ap-northeast-1.amazonaws.com/InstallSQLiteStudio-3.2.1.dmg)

## 條件邏輯

## 條件除了能夠運用在 `WHERE` 篩選資料，另外一個常見的應用場景是使用 `CASE` 衍生計算欄位

```sql
SELECT CASE WHEN condition_1 THEN result_1
            WHEN condition_2 THEN result_2
            ...
            ELSE result_else END AS alias;
```

## 使用布林表示電影是否在千禧年之後上映

In [2]:
SELECT title,
       release_year >= 2000 AS after_millennium_bool
  FROM movies
 LIMIT 5;

title,after_millennium_bool
The Shawshank Redemption,0
The Godfather,0
The Godfather: Part II,0
The Dark Knight,1
12 Angry Men,0


## 使用文字表示電影是否在千禧年之後上映

In [3]:
SELECT title,
       release_year,
       CASE WHEN release_year >= 2000 THEN 'true'
            ELSE 'false' END AS after_millennium
  FROM movies
 LIMIT 5;

title,release_year,after_millennium
The Shawshank Redemption,1994,false
The Godfather,1972,false
The Godfather: Part II,1974,false
The Dark Knight,2008,true
12 Angry Men,1957,false


## 使用文字將電影的評等分為三類

In [4]:
SELECT DISTINCT CASE WHEN rating >= 8.8 THEN 'Super high'
                     WHEN rating >= 8.5 THEN 'Medium high'
                     ELSE 'high' END AS rating_text
  FROM movies;

rating_text
Super high
Medium high
high


## 如果在某個觀測值發生 `condition_1` 與 `condition_2` 都判斷為真的情況下，會以 `result_1` 表示

```sql
SELECT CASE WHEN condition_1 THEN result_1
            WHEN condition_2 THEN result_2
            ...
            ELSE result_else END AS alias;
```

## 當條件沒有互斥的時候，撰寫 `CASE` 要特別注意順序

In [5]:
SELECT DISTINCT CASE WHEN rating >= 8.5 THEN 'Medium high' -- 先判斷 Medium high 結果不如預期
                     WHEN rating >= 8.8 THEN 'Super high'
                     ELSE 'high' END AS rating_text
  FROM movies;

rating_text
Medium high
high


##  如果不想特別注意順序，可以將條件設計為互斥

In [6]:
SELECT DISTINCT CASE WHEN rating >= 8.5 AND rating < 8.8 THEN 'Medium high'
                     WHEN rating >= 8.8 THEN 'Super high'
                     ELSE 'high' END AS rating_text
  FROM movies;

rating_text
Super high
Medium high
high


## 函數

## Function，中文翻譯為函數或者函式，在資料分析和程式語言中都扮演舉足輕重的角色

函數是預先被定義好的運算處理邏輯，透過它的作用，能夠將「輸入」對應為「輸出」，進而完成計算數值與操作文字等任務。

## 函數的運作有五個組成：

1. 函數的名稱。
2. 輸入。
3. 參數。
4. 運算處理邏輯。
5. 輸出。

## 以買珍珠奶茶為例

![](images/pearl_milk_tea.jpg)

來源：[Google Search](https://www.google.com/search?q=%E7%8F%8D%E7%8F%A0%E5%A5%B6%E8%8C%B6&sxsrf=ALeKk03IFHCE14mwz0z5yVStCaGHvED-8g:1617364912723&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjZ9eDowd_vAhVgxosBHdTsC_MQ_AUoAXoECAEQAw&biw=1037&bih=1448&dpr=1)

## 以一個常用的文字操作函數 `SUBSTR` 為例

In [7]:
SELECT 'Tony Stark' AS full_name,
       SUBSTR('Tony Stark', 1, 4) AS first_name,
       SUBSTR('Tony Stark', 6, 5) AS last_name;

full_name,first_name,last_name
Tony Stark,Tony,Stark


## 函數可依照功能分為兩大類：

- 通用函數（Universal functions）
    - 轉換資料類型。
    - 計算數值。
    - 操作文字。
    - 操作日期時間。
- 聚合函數（Aggregate functions）

## 通用函數與聚合函數的不同在於其所作用的維度

- 通用函數作用在「水平」方向。
- 聚合函數作用在「垂直」方向。

## 作用在「水平」方向的通用函數

效果類似「衍生計算欄位」，差別在於一個是以函數輸出衍生計算欄位，一個則是以運算符生成衍生計算欄位。

![mutate](images/mutate.png)

## 作用在「垂直」方向的聚合函數

![aggregate](images/aggregate.png)

## 通用函數的特徵：每個觀測值對應一個輸出

In [8]:
SELECT rating,
       ROUND(rating) AS rounded_rating
  FROM movies
 LIMIT 5;

rating,rounded_rating
9.3,9.0
9.2,9.0
9,9.0
9,9.0
9,9.0


## 聚合函數的特徵：整欄變數對應一個輸出

In [9]:
SELECT AVG(rating) AS avg_rating
  FROM movies;

avg_rating
8.30519999999997


## 常見的通用函數又可以細分為四種類型

- **轉換資料類型**。
- 計算數值。
- 操作文字。
- 操作日期時間。

## 使用 `CAST` 函數可以將查詢結果的資料類型轉換為指定資料類型

```sql
CAST(data AS data_type)
```

In [10]:
SELECT release_year,
       TYPEOF(release_year) AS typeof_release_year,
       CAST(release_year AS float) AS cast_release_year,
       TYPEOF(CAST(release_year AS float)) AS float_release_year
  FROM movies
 LIMIT 1;

release_year,typeof_release_year,cast_release_year,float_release_year
1994,integer,1994.0,real


## 使用 `COALESCE` 函數可以將空值（或稱遺漏值）轉換為指定常數

```sql
COALESCE(NULL, replacement)
```

In [11]:
SELECT NULL AS a_null,
       TYPEOF(NULL) AS typeof_null,
       COALESCE(NULL, '遺漏值') AS text_null,
       TYPEOF(COALESCE(NULL, '遺漏值')) AS typeof_text_null;

a_null,typeof_null,text_null,typeof_text_null
NULL,null,遺漏值,text


## 常見的通用函數又可以細分為四種類型（續）

- 轉換資料類型。
- **計算數值**。
- 操作文字。
- 操作日期時間。

## 使用 `ROUND` 函數可以調整查詢結果的小數點位數

```sql
ROUND(REAL, n_digits)
```

In [12]:
SELECT ROUND(AVG(rating), 2) AS average_rating
  FROM movies;

average_rating
8.31


## 常見的通用函數又可以細分為四種類型（續）

- 轉換資料類型。
- 計算數值。
- **操作文字**。
- 操作日期時間。

## 使用 `LENGTH` 函數可以計算文字中有幾個字元，包含空格、標點符號

```sql
LENGTH(TEXT)
```

In [13]:
SELECT title,
       LENGTH(title) AS length_of_movie_title
  FROM movies
 LIMIT 5;

title,length_of_movie_title
The Shawshank Redemption,24
The Godfather,13
The Godfather: Part II,22
The Dark Knight,15
12 Angry Men,12


## 使用 `INSTR` 函數可回傳文字中指定特徵的開始位置

```sql
INSTR(TEXT, pattern)
```

In [14]:
SELECT INSTR(title, 'Dark Knight') AS dark_knight_position,
       INSTR(title, 'Batman') AS batman_position
  FROM movies
 LIMIT 5;

dark_knight_position,batman_position
0,0
0,0
0,0
5,0
0,0


In [15]:
SELECT *
  FROM movies
 WHERE INSTR(title, 'Dark Knight') > 0 OR
       INSTR(title, 'Batman') > 0;

id,title,release_year,rating,director,runtime
4,The Dark Knight,2008,9,Christopher Nolan,152
71,The Dark Knight Rises,2012,8.4,Christopher Nolan,164
127,Batman Begins,2005,8.2,Christopher Nolan,140


## 使用 `LOWER` 與 `UPPER` 函數可以調整英文的大小寫

```sql
LOWER(TEXT)
UPPER(TEXT)
```

In [16]:
SELECT UPPER(title) AS upper_title,
       LOWER(title) AS lower_title
  FROM movies
 LIMIT 5;

upper_title,lower_title
THE SHAWSHANK REDEMPTION,the shawshank redemption
THE GODFATHER,the godfather
THE GODFATHER: PART II,the godfather: part ii
THE DARK KNIGHT,the dark knight
12 ANGRY MEN,12 angry men


## 常見的通用函數又可以細分為四種類型（續）

- 轉換資料類型。
- 計算數值。
- 操作文字。
- **操作日期時間**。

## 標準的日期、時間與日期時間格式：ISO8601

- 日期 `YYYY-MM-DD`
- 時間 `HH:MM:SS`
- 日期時間 `YYYY-MM-DD HH:MM:SS`

## 使用 `DATE()` 函數顯示系統日期

In [17]:
SELECT DATE('now') AS date_of_today;

date_of_today
2021-12-20


## 使用 `TIME()` 函數顯示系統時間

In [18]:
SELECT TIME('now', 'localtime') AS time_of_now;

time_of_now
19:37:50


## 使用 `DATETIME()` 函數顯示系統日期時間

In [19]:
SELECT DATETIME('now', 'localtime') AS date_time_of_now;

date_time_of_now
2021-12-20 19:37:50


## 使用 `STRFTIME` 函數調整日期、時間或日期時間的顯示格式

```sql
STRFTIME(format, DATE/TIME/DATETIME)
```

## 常見的日期與日期時間格式參數

- `%d`：二位數的日（01-31）
- `%j`：一年中的第幾天（001-366）
- `%m`：二位數的月（01-12）
- `%w`：一星期中的第幾天（0-6）
- `%W`：一年中的第幾週（00-53）
- `%Y`：四位數的年（0000-9999）

In [20]:
SELECT DATE('now') AS date_of_today,
       STRFTIME('%d', DATE('now')) AS day_part,
       STRFTIME('%j', DATE('now')) AS year_day_format,
       STRFTIME('%m', DATE('now')) AS month_part,
       STRFTIME('%w', DATE('now')) AS weekday,
       STRFTIME('%W', DATE('now')) AS nth_week,
       STRFTIME('%Y', DATE('now')) AS year_part;

date_of_today,day_part,year_day_format,month_part,weekday,nth_week,year_part
2021-12-20,20,354,12,1,51,2021


## SQLite 通用函數與操作日期時間函數的官方文件

- [Built-In Scalar SQL Functions](https://www.sqlite.org/lang_corefunc.html)
- [Date And Time Functions](https://www.sqlite.org/lang_datefunc.html)

## 常見的聚合函數

- `AVG(column_name)`：計算變數的平均數
- `COUNT(column_name)`：計算變數的「非」遺漏值數
- `COUNT(*)`：計算資料表的觀測值數
- `MAX(column_name)`：計算變數的最大值
- `MIN(column_name)`：計算變數的最小值
- `SUM(column_name)`：計算變數的加總

In [21]:
SELECT AVG(rating) AS avg_rating
  FROM movies;

avg_rating
8.30519999999997


In [22]:
SELECT COUNT(*) AS number_of_rows
  FROM movies;

number_of_rows
250


In [23]:
SELECT COUNT(NULL) AS number_of_rows;

number_of_rows
0


In [24]:
SELECT MAX(release_year) AS max_release_year
  FROM movies;

max_release_year
2021


In [25]:
SELECT MIN(release_year) AS min_release_year
  FROM movies;

min_release_year
1921


## SQLite 聚合函數的官方文件

[Built-in Aggregate Functions](https://www.sqlite.org/lang_aggfunc.html)

## 分組與聚合結果篩選

## 分組 `GROUP BY` 的功能可以視為 `DISTINCT` 與 `ORDER BY` 兩者同時作用

```sql
SELECT column_names
  FROM table_name
 GROUP BY column_names;
```

## `DISTINCT` 與 `ORDER BY` 兩者同時作用

In [26]:
SELECT DISTINCT director AS distinct_director
  FROM movies
 ORDER BY director;

distinct_director
Aamir Khan
Adam Elliot
Akira Kurosawa
Alejandro G. Iñárritu
Alfred Hitchcock
Andrei Tarkovsky
Andrew Stanton
Anthony Russo
Anurag Kashyap
Asghar Farhadi


## 使用分組 `GROUP BY`

In [27]:
SELECT director AS distinct_director
  FROM movies
 GROUP BY director;

distinct_director
Aamir Khan
Adam Elliot
Akira Kurosawa
Alejandro G. Iñárritu
Alfred Hitchcock
Andrei Tarkovsky
Andrew Stanton
Anthony Russo
Anurag Kashyap
Asghar Farhadi


## 在「函數」的章節，我們介紹過一種「用來彙總資訊」的函數，稱為聚合函數（Aggregate functions）

單獨使用聚合函數的時候，是將一整欄變數的資訊彙總後輸出。

In [28]:
SELECT AVG(rating) AS avg_rating  -- 所有電影的平均評等
  FROM movies;

avg_rating
8.30519999999997


## 假如現在希望計算不同每一年的電影平均評等，現在我們會怎麼做？

- 先知道有哪些上映年份。
- 篩選不同上映年份，一一計算平均評等。

In [29]:
SELECT DISTINCT release_year --先知道有哪些上映年份
  FROM movies;

release_year
1994
1972
1974
2008
1957
1993
2003
1966
2001
1999


## 這不是一個聰明的辦法

In [30]:
SELECT COUNT(DISTINCT release_year) AS number_of_dist_release_year
  FROM movies;

number_of_dist_release_year
85


## 結合聚合函數與 `GROUP BY` 可以便捷地完成分組聚合

```sql
SELECT AGGREGATE_FUNCTION(column_names) AS alias
  FROM table_name
 GROUP BY column_names;
```

![group by](images/group_by.png)

In [31]:
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM movies
 GROUP BY release_year;

release_year,avg_rating
1921,8.3
1924,8.2
1925,8.2
1926,8.1
1927,8.3
1928,8.1
1931,8.4
1934,8.1
1936,8.5
1939,8.1


## 用來篩選資料的兩種方式

1. 作用在「觀測值」的 `WHERE`
2. 作用在「分組聚合結果」的 `HAVING`

## 作用在「分組聚合結果」的 `HAVING`

```sql
SELECT AGGREGATE_FUNCTION(column_names) AS alias
  FROM table_name
 GROUP BY column_names
HAVING conditions;
```

In [32]:
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM movies
 GROUP BY release_year
HAVING avg_rating >= 8.5;

release_year,avg_rating
1936,8.5
1946,8.6
1972,9.2
1974,8.55
1977,8.6
1990,8.7
1991,8.55
1994,8.8
1999,8.5
2008,8.5


## 子查詢

## 在「函數」的章節中，我們看過函數包含著其他函數的使用方法

前一個函數的輸出，成為下一個函數的輸入。

In [33]:
SELECT ROUND(AVG(rating)) AS avg_rating 
  FROM movies;

avg_rating
8.0


## 如果一段 SQL 中包含著另外一段的 SQL，這樣的 SQL 結構就被稱為子查詢（Subquery）

## 常見的子查詢應用情境

## 在什麼樣的情境下我們會想使用子查詢呢？

1. 查詢的篩選條件必須要先做一個查詢才能得知。
2. 查詢的計算內容必須要先做一個查詢才能得知。
3. 查詢所需要的資料來自不同資料表。
4. 將查詢的結果作為一個資料表使用。

## 情境一：查詢的篩選條件必須要先做一個查詢才能得知

我們想知道 `imdb` 資料庫的 `movies` 資料表中片長最短的電影是哪一部。

```sql
SELECT *
  FROM movies
 WHERE runtime = MIN(runtime); -- 這個查詢會得到錯誤，因為聚合函數不能夠寫在 WHERE 後
```

## 我們要分兩次查詢來完成

- 先查詢「最短」的片長是幾分鐘。
- 再依據前一個查詢結果作為篩選條件。

In [34]:
SELECT MIN(runtime) AS minimum_runtime  -- 先查詢「最短」的片長是幾分鐘。
  FROM movies;

minimum_runtime
45


In [35]:
SELECT *
  FROM movies
 WHERE runtime = 45;  -- 再依據前一個查詢結果作為篩選條件。

id,title,release_year,rating,director,runtime
195,Sherlock Jr.,1924,8.2,Buster Keaton,45


## 面對情境一，我們可以應用子查詢將一段 SQL 查詢結果作為條件

```sql
SELECT column_names
  FROM table_name
 WHERE conditions (Another SELECT statement);
```

In [36]:
SELECT MIN(runtime) AS minimum_runtime  -- 先查詢「最短」的片長是幾分鐘。
  FROM movies;

SELECT *
  FROM movies
 WHERE runtime = 45;  -- 再依據前一個查詢結果作為篩選條件。

minimum_runtime
45


## 情境二：查詢的計算內容必須要先做一個查詢才能得知

我們想知道 `imdb` 資料庫的 `movies` 資料表中，在千禧年之後上映的電影佔比為多少？

## 我們要分兩次查詢來完成

- 先查詢在千禧年之後上映的電影有幾部。
- 再依據前一個查詢結果作為計算內容。

In [37]:
SELECT COUNT(*) AS number_of_movies
  FROM movies
 WHERE release_year >= 2000;  -- 先查詢在千禧年之後上映的電影有幾部。

number_of_movies
101


In [38]:
SELECT 101 / CAST(COUNT(*) AS REAL) AS millennium_percentage  -- 再依據前一個查詢結果作為計算內容。
  FROM movies;

millennium_percentage
0.404


## 面對情境二，我們可以應用子查詢將一段 SQL 查詢結果作為計算內容

```sql
SELECT (Another SELECT statement) AS alias
  FROM table_name;
```

In [39]:
SELECT COUNT(*) AS number_of_movies
  FROM movies
 WHERE release_year >= 2000;  -- 先查詢在千禧年之後上映的電影有幾部。

SELECT 101 / CAST(COUNT(*) AS REAL) AS millennium_percentage  -- 再依據前一個查詢結果作為計算內容。
  FROM movies;

number_of_movies
101


## 情境三：查詢所需要的資料來自不同的資料表

我們想知道在 `imdb` 資料庫中，哪幾部電影 Tom Hanks 有演出？

## 我們要分三次查詢來完成

- 先從 `actors` 資料表查詢 Tom Hanks 的演員編號是多少。
- 再依據前一個查詢結果去 `casting` 資料表查詢。
- 再依據前一個查詢結果去 `movies` 資料表查詢。

In [40]:
SELECT id
  FROM actors
 WHERE name = 'Tom Hanks';  -- 先從 actors 資料表查詢 Tom Hanks 的演員編號是多少。

id
2865


In [41]:
SELECT movie_id
  FROM casting
 WHERE actor_id = 2865;  -- 再依據前一個查詢結果去 casting 資料表查詢。

movie_id
12
26
27
82
112
189


In [42]:
SELECT *
  FROM movies
 WHERE id IN (12, 26, 27, 82, 112, 189);  -- 再依據前一個查詢結果去 movies 資料表查詢。

id,title,release_year,rating,director,runtime
12,Forrest Gump,1994,8.8,Robert Zemeckis,142
26,Saving Private Ryan,1998,8.6,Steven Spielberg,169
27,The Green Mile,1999,8.6,Frank Darabont,189
82,Toy Story,1995,8.3,John Lasseter,81
112,Toy Story 3,2010,8.2,Lee Unkrich,103
189,Catch Me If You Can,2002,8.1,Steven Spielberg,141


## 面對情境三，我們可以應用子查詢將一段 SQL 查詢結果作為條件

```sql
SELECT column_names
  FROM table_name
 WHERE conditions (Another SELECT statement);
```

In [43]:
SELECT id
  FROM actors
 WHERE name = 'Tom Hanks';  -- 先從 actors 資料表查詢 Tom Hanks 的演員編號是多少。
 
SELECT movie_id
  FROM casting
 WHERE actor_id = 2865;  -- 再依據前一個查詢結果去 casting 資料表查詢。
 
SELECT *
  FROM movies
 WHERE id IN (12, 26, 27, 82, 112, 189);  -- 再依據前一個查詢結果去 movies 資料表查詢。

id
2865


## 情境四，應用子查詢將一段 SQL 查詢結果作為資料表

```sql
SELECT column_names
  FROM (SELECT column_names FROM table_name) AS alias;
```

In [44]:
SELECT *
  FROM (SELECT release_year,
               MAX(rating) AS max_rating
          FROM movies
         GROUP BY release_year) AS max_rating_each_year
 ORDER BY release_year DESC
 LIMIT 10;

release_year,max_rating
2021,8.3
2020,8.5
2019,8.6
2018,8.5
2017,8.4
2016,8.4
2015,8.2
2014,8.6
2013,8.4
2012,8.4


## 關聯資料表

## 在「子查詢 」章節中，我們知道子查詢常見的應用情境有：

1. 查詢的篩選條件必須要先做一個查詢才能得知。
2. 查詢的計算內容必須要先做一個查詢才能得知。
3. 查詢所需要的**資料來自不同資料表**。
4. 將查詢的結果作為一個資料表使用。

## 這意味著我們的查詢範圍開始由「單個」資料表擴展至「多個」資料表

- 子查詢。
- 關聯資料表。

## 關聯式資料庫（Relational Database）的核心精神

- 可以設計資料表彼此之間的「關聯」特性。
- 讓資料表彼此之間的「重複性」降低。

## 資料表能夠從兩個維度關聯

- 垂直：讓資料表以垂直方向對觀測值（列）進行集合運算。
- 水平：讓資料表以水平方向對變數（欄）進行集合運算。

## 以 `UNION` 垂直關聯

```sql
A SELECT statement
UNION
Another SELECT statement
```

## 垂直關聯簡單來說，就是結合觀測值

![union](images/union.png)

In [45]:
SELECT director AS my_favorites
  FROM movies
 WHERE director IN ('Christopher Nolan', 'Steven Spielberg')
 UNION
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Leonardo DiCaprio');

my_favorites
Christopher Nolan
Leonardo DiCaprio
Steven Spielberg
Tom Hanks


## 使用 `UNION` 的注意事項

- 垂直關聯的欄位數要相同。
- 垂直關聯的 SQL 若有使用到 `ORDER BY` 只能放在 `UNION` 之後。
- 垂直關聯的重複觀測值會被省略。

## 垂直關聯的欄位數要相同

```sql
SELECT director AS my_favorites
  FROM movies
 WHERE director IN ('Christopher Nolan', 'Steven Spielberg')
 UNION
SELECT name,
       id  -- 垂直關聯的欄位數不相同
  FROM actors
 WHERE name IN ('Tom Hanks', 'Leonardo DiCaprio');
```

## 垂直關聯的 SQL 若有使用到 `ORDER BY` 只能放在 `UNION` 之後

```sql
SELECT director AS my_favorites
  FROM movies
 WHERE director IN ('Christopher Nolan', 'Steven Spielberg')
 ORDER BY my_favorites
 UNION
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Leonardo DiCaprio');
```

## 垂直關聯的重複觀測值會被省略

In [46]:
SELECT director AS my_favorites
  FROM movies
 WHERE director IN ('Christopher Nolan', 'Steven Spielberg');
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Leonardo DiCaprio')
 ORDER BY my_favorites;

my_favorites
Christopher Nolan
Steven Spielberg
Christopher Nolan
Steven Spielberg
Christopher Nolan
Christopher Nolan
Christopher Nolan
Steven Spielberg
Christopher Nolan
Steven Spielberg


## 若希望保留重複觀測值，改使用 `UNION ALL`

```sql
A SELECT statement
UNION ALL
Another SELECT statement
```

In [47]:
SELECT director AS my_favorites
  FROM movies
 WHERE director IN ('Christopher Nolan', 'Steven Spielberg')
 UNION ALL
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Leonardo DiCaprio')
 ORDER BY my_favorites;

my_favorites
Christopher Nolan
Christopher Nolan
Christopher Nolan
Christopher Nolan
Christopher Nolan
Christopher Nolan
Christopher Nolan
Leonardo DiCaprio
Steven Spielberg
Steven Spielberg


## 水平關聯

## 學習資料庫的實體關係圖（ER Diagram, Entity Relationship Diagram） 

- 體會關聯式資料庫的核心精神。
- 暸解資料表之間透過如同「橋樑」般存在的**結合鍵**（Join Key）水平關聯。

## `imdb` 資料庫的實體關係圖

![erd_imdb](images/imdb_erd.png)

## 以 `JOIN` 水平關聯

因為是「水平」關聯，在 `FROM` 後的資料表被稱為「左表格」、`JOIN` 後的資料表被稱為「右表格」。

```sql
SELECT left_table.column_names,
       right_table.column_names
  FROM left_table
  JOIN right_table
    ON left_table.join_key = right_table.join_key;
```

## 水平關聯簡單來說，就是結合變數

![join](images/join.png)

## 以 `JOIN` 水平關聯 `movies` 與 `casting`

In [48]:
SELECT movies.title,
       casting.actor_id
  FROM movies  -- 左表格
  JOIN casting -- 右表格
    ON movies.id = casting.movie_id
 WHERE movies.title = 'The Shawshank Redemption';

title,actor_id
The Shawshank Redemption,2853
The Shawshank Redemption,2097
The Shawshank Redemption,333
The Shawshank Redemption,3029
The Shawshank Redemption,533
The Shawshank Redemption,1042
The Shawshank Redemption,1923
The Shawshank Redemption,1319
The Shawshank Redemption,1370
The Shawshank Redemption,1724


## 以 `JOIN` 水平關聯 `movies`、`casting` 與 `actors`

In [49]:
SELECT movies.title,
       casting.ord,
       actors.name
  FROM movies   -- 左表格
  JOIN casting  -- 右一表格
    ON movies.id = casting.movie_id
  JOIN actors   -- 右二表格
    ON casting.actor_id = actors.id
 WHERE movies.title = 'The Shawshank Redemption';

title,ord,name
The Shawshank Redemption,1,Tim Robbins
The Shawshank Redemption,2,Morgan Freeman
The Shawshank Redemption,3,Bob Gunton
The Shawshank Redemption,4,William Sadler
The Shawshank Redemption,5,Clancy Brown
The Shawshank Redemption,6,Gil Bellows
The Shawshank Redemption,7,Mark Rolston
The Shawshank Redemption,8,James Whitmore
The Shawshank Redemption,9,Jeffrey DeMunn
The Shawshank Redemption,10,Larry Brandenburg


## 使用 `JOIN` 水平關聯資料表時的注意事項

- 養成為水平關聯的資料表取別名並在欄位名稱前註明清楚的好習慣。
- 除了實體資料表亦可使用子查詢生成的結果作為關聯來源。
- 預設保留左表格與右表格**交集**的觀測值。

## 養成為水平關聯的資料表取別名並在欄位名稱前註明清楚的好習慣

舉例來說，在 `movies` 與 `actors` 兩個資料表中都有 `id` 欄位。

- `movies.id` 指的是電影編號。
- `actors.id` 指的是演員編號。

## 如果沒有註明 id 來自哪個資料表會產生錯誤

```sql
SELECT title,
       ord,
       name
  FROM movies
  JOIN casting
    ON id = movie_id
  JOIN actors
    ON actor_id = id
 WHERE title = 'The Shawshank Redemption';
```

## 除了實體資料表亦可使用子查詢生成的結果作為關聯來源

In [50]:
SELECT subquery_movies.title,
       casting.ord,
       actors.name
  FROM (SELECT id,
               title,
               director
          FROM movies
         WHERE title IN ('The Shawshank Redemption', 'Forrest Gump')) AS subquery_movies -- 左表格
  JOIN casting  -- 右一表格
    ON subquery_movies.id = casting.movie_id
  JOIN actors   -- 右二表格
    ON casting.actor_id = actors.id
 ORDER BY ord;

title,ord,name
The Shawshank Redemption,1,Tim Robbins
Forrest Gump,1,Tom Hanks
The Shawshank Redemption,2,Morgan Freeman
Forrest Gump,2,Rebecca Williams
The Shawshank Redemption,3,Bob Gunton
Forrest Gump,3,Sally Field
The Shawshank Redemption,4,William Sadler
Forrest Gump,4,Michael Conner Humphreys
The Shawshank Redemption,5,Clancy Brown
Forrest Gump,5,Harold G. Herthum


## 預設保留左表格與右表格交集的觀測值

In [51]:
SELECT two_movies.title,
       two_castings.actor_id
  -- 左表格是刺激1995與阿甘正傳的電影資料
  FROM (SELECT *
          FROM movies
         WHERE title IN ('The Shawshank Redemption', 'Forrest Gump')) AS two_movies
  -- 右表格是刺激1995與黑暗騎士的演員名單資料
  JOIN (SELECT *
          FROM casting
         WHERE movie_id IN (1, 4)) AS two_castings
    ON two_movies.id = two_castings.movie_id
 ORDER BY two_movies.title;

title,actor_id
The Shawshank Redemption,2853
The Shawshank Redemption,2097
The Shawshank Redemption,333
The Shawshank Redemption,3029
The Shawshank Redemption,533
The Shawshank Redemption,1042
The Shawshank Redemption,1923
The Shawshank Redemption,1319
The Shawshank Redemption,1370
The Shawshank Redemption,1724


## 若希望保留以左表格為主的觀測值，改使用 `LEFT JOIN`

```sql
SELECT left_table.column_names,
       right_table.column_names
  FROM table_name AS left_table
  LEFT JOIN table_name AS right_table
    ON left_table.join_key = right_table.join_key;
```

In [52]:
SELECT two_movies.title,
       two_castings.actor_id
  -- 左表格是刺激1995與阿甘正傳的電影資料
  FROM (SELECT *
          FROM movies
         WHERE title IN ('The Shawshank Redemption', 'Forrest Gump')) AS two_movies
  -- 右表格是刺激1995與黑暗騎士的名單資料
  LEFT JOIN (SELECT *
               FROM casting
              WHERE movie_id IN (1, 4)) AS two_castings
    ON two_movies.id = two_castings.movie_id
 ORDER BY two_movies.title;

title,actor_id
Forrest Gump,NULL
The Shawshank Redemption,333
The Shawshank Redemption,363
The Shawshank Redemption,533
The Shawshank Redemption,659
The Shawshank Redemption,1042
The Shawshank Redemption,1319
The Shawshank Redemption,1370
The Shawshank Redemption,1530
The Shawshank Redemption,1553


## 若希望保留以右表格為主的觀測值，互換左右表格

或者在使用其他關聯式資料庫管理系統時檢查是否有 `RIGHT JOIN` 功能。

In [53]:
SELECT two_movies.title,
       two_castings.actor_id
  -- 左表格是刺激1995與黑暗騎士的名單資料
  FROM (SELECT *
               FROM casting
              WHERE movie_id IN (1, 4)) AS two_castings
  -- 右表格是刺激1995與阿甘正傳的電影資料
  LEFT JOIN (SELECT *
               FROM movies
              WHERE title IN ('The Shawshank Redemption', 'Forrest Gump')) AS two_movies
    ON two_castings.movie_id = two_movies.id
 ORDER BY two_movies.title;

title,actor_id
NULL,502
NULL,1155
NULL,2
NULL,2015
NULL,1855
NULL,985
NULL,2097
NULL,2095
NULL,2520
NULL,528


## 若希望保留左右表格的觀測值，加入 `UNION`

或者在使用其他關聯式資料庫管理系統時檢查是否有 `FULL JOIN` 功能。

In [54]:
SELECT two_movies.title,
       two_castings.actor_id
  -- 左表格是刺激1995與阿甘正傳的電影資料
  FROM (SELECT *
          FROM movies
         WHERE title IN ('The Shawshank Redemption', 'Forrest Gump')) AS two_movies
  -- 右表格是刺激1995與黑暗騎士的名單資料
  LEFT JOIN (SELECT *
               FROM casting
              WHERE movie_id IN (1, 4)) AS two_castings
    ON two_movies.id = two_castings.movie_id
 UNION
SELECT two_movies.title,
       two_castings.actor_id
  -- 左表格是刺激1995與黑暗騎士的名單資料
  FROM (SELECT *
               FROM casting
              WHERE movie_id IN (1, 4)) AS two_castings
  -- 右表格是刺激1995與阿甘正傳的電影資料
  LEFT JOIN (SELECT *
               FROM movies
              WHERE title IN ('The Shawshank Redemption', 'Forrest Gump')) AS two_movies
    ON two_castings.movie_id = two_movies.id
 ORDER BY two_movies.title;

title,actor_id
NULL,2
NULL,179
NULL,482
NULL,502
NULL,528
NULL,838
NULL,985
NULL,1155
NULL,1855
NULL,2015


## 若希望保留以右表格為主的觀測值，互換左右表格

In [ ]:
/*截至目前學起來的 SQL 有哪些？
SQL 寫作順序必須遵從標準 SQL 的規定。*/
SELECT column_names     -- 選擇哪些欄位
  FROM left_table       -- 從哪個資料庫的資料表
  JOIN right_table      -- 與哪個資料表水平關聯
    ON left_table.join_key = right_table.join_key
 WHERE conditions       -- 篩選哪些觀測值
 GROUP BY column_names  -- 指定依照哪個變數分組
HAVING conditions       -- 篩選哪些分組聚合的結果
 UNION SELECT statement -- 與哪段 SQL 垂直關聯
 ORDER BY column_names  -- 指定依照哪個變數排序
 LIMIT m;               -- 查詢結果顯示前 m 列就好